In [1]:
import os
from tqdm import tqdm, trange
os.chdir("../")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
pwd

'/home/YiVal_test'

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    #AutoModelForQuestionAnswering,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [4]:
import multiprocessing
# The model that you want to train from the Hugging Face hub
model_name = "tiiuae/falcon-40b"

# The instruction dataset to use
dataset_name = "cais/mmlu"
dataset_config = {"version": "all"}
# Fine-tuned model name
new_model = "falcon-40b-mmlu"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = 20000

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 50

################################################################################
# PEFT Settings
################################################################################
target_modules = [
    'query_key_value',
    'dense',
    'dense_h_to_4h',
    'dense_4h_to_h',
    'lm_head'
]

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

response_template = " ### Answer:"
# Load the entire model on the GPU 0
device_map = {"": 0}

In [5]:
# Formatting function of dataset MMLU
def process_example(example):
    output_texts = []
    for i in range(len(example['answer'])):
    #print(example['answer'])
        text = f'''### Question: {example['question'][i]}\n 
            ### Choices: {example['choices'][i]}\n 
            # ### Answer: {example['answer'][i]}'''
        output_texts.append(text)
    return output_texts

def formatting_funcion(examples):
    output_texts = []
    print(examples)   
    #with multiprocessing.Pool() as pool:
    #    mult_res = [pool.apply_async(process_example, (e,)) for e in examples]
    #    for (i, res) in enumerate(mult_res):
    #        output_texts.append(res.get())
    for e in examples:
        output_texts.append(process_example(e))    
    return output_texts

def custom_formatting(example):
    output_text = f'''### Question: {example['question']}\n 
                    ### Choices: {example['choices']}\n 
                    ### Answer: {example['answer']}'''
    return {"text":output_text}

# Load dataset (you can process it here)
dataset = load_dataset(dataset_name,  name = "all",  split="auxiliary_train", streaming=True)
dataset = dataset.with_format("torch")
formatted_dataset = dataset.map(custom_formatting, batched=True)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1



Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

/home/YiVal_test/.cache/pypoetry/virtualenvs/yival-0EQAwXAc-py3.10/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
formatted_dataset = formatted_dataset.remove_columns(dataset.column_names)
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, truncation=True, padding=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="causal_lm",
    target_modules=target_modules,
)

tokenized_dataset = formatted_dataset.map(lambda examples: tokenizer(examples["text"]), batched=True)

In [7]:
print(formatted_dataset.column_names)

None


In [11]:

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    #group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    remove_unused_columns=False
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=peft_config,
    #formatting_func=custom_formatting,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


/home/YiVal_test/.cache/pypoetry/virtualenvs/yival-0EQAwXAc-py3.10/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


/home/YiVal_test/.cache/pypoetry/virtualenvs/yival-0EQAwXAc-py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [12]:
tokenized_dataset

In [13]:

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

ValueError: Column lengths mismatch: columns ['text'] have length [1226805] while question has length 1000.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

Reusing TensorBoard on port 6006 (pid 53467), started 1 day, 18:32:45 ago. (Use '!kill 53467' to kill it.)

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/home/YiVal_test/.cache/pypoetry/virtualenvs/yival-0EQAwXAc-py3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/home/YiVal_test/.cache/pypoetry/virtualenvs/yival-0EQAwXAc-py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<s>[INST] What is a large language model? [/INST]  A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a vast amount of natural language data, such as text, to learn the patterns and relationships between words and phrases. The size of an LLM refers to the amount of data it has been trained on, which can range from millions to billions of words.

LLMs are typically deep neural network models that use techniques such as recurrent neural networks (RNNs) or transformer architectures to learn the complexities of natural language. They can be used for a variety of natural language processing tasks, such as language translation, text summarization, question answering, and language generation.

Some examples of large language models include GPT-3, BERT, and RoBERTa. These models have been trained on vast amounts of text data and have demonstrated the ability to generate


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

: 